Task: We are interested only in the electrical potentials of neurons (not in the molecular concentrations). A simple IF just sums (integrate) the spikes received increasing the membrane potential and fires if the potential value is greater than a threshold.

Apply the specification hierarchy and propose a state transition diagram.
Detail the variable mathematical definitions
Parameters: (∆,θ), ∆ is the firing delay once the threshold iscrossed, and θ = 2 is the threshold.



The state transition Diagram:

R $\xrightarrow[\text{}]{\text{θ}}$ F

R: resting state, neuron is at rest with a membrane potential of 0.

F: firing state, neuron fires an output spike once the membrane potential exceeds the threshold θ.


- ∆: firing delay, the time delay from crossing the threshold to firing an output spike.
Firing delay is a constant value representing the time delay from crossing the threshold to firing an output spike.

- θ: threshold, the membrane potential threshold that triggers the neuron to fire.
Threshold is a constant value set to 2, representing the membrane potential threshold that triggers the neuron to fire.

In [3]:
import numpy as np

#The Neuron class represents an individual neuron 
#with its respective weights, threshold, firing delay, membrane potential, activation state, and time since last firing.
class Neuron:
    def __init__(self, weights, threshold, firing_delay):
        self.weights = weights
        self.threshold = threshold
        self.firing_delay = firing_delay
        self.membrane_potential = 0.0  #We initialize the membrane potential to 0.
        self.activation = np.zeros_like(threshold)  #We initialize the activation state to 0.
        self.time_since_firing = np.zeros_like(firing_delay)  #We initialize the time since firing to 0.

    def integrate_fire(self, inputs):
        #We define function integrate_fire, which integrate inputs by adding the dot product of inputs and weights to the membrane potential.
        self.membrane_potential += np.dot(inputs, self.weights)
        #Here we check if neuron is not already activated.
        threshold_exceeded = np.logical_and(self.activation == 0, self.membrane_potential >= self.threshold)
        #Then, we update the activation state and reset the membrane potential if the threshold is exceeded.
        self.activation = np.where(threshold_exceeded, 1, 0)
        self.membrane_potential = np.where(threshold_exceeded, 0.0, self.membrane_potential)
        #We update the time since firing for activated neurons, and reset for others.
        self.time_since_firing = np.where(self.activation == 0, self.time_since_firing + 1, 0)

    def is_ready_to_fire(self):
        #We look if the neuron is activated and has surpassed the firing delay.
        return np.logical_and(self.activation == 1, self.time_since_firing >= self.firing_delay)

In [4]:
#The NeuralNetwork class represents the neural network 
#with its layers, weights, thresholds, and firing delay.

class NeuralNetwork:
    def __init__(self, input_neurons, hidden_neurons, output_neurons, threshold_hidden, threshold_output, firing_delay):
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons
        self.threshold_hidden = threshold_hidden
        self.threshold_output = threshold_output
        self.firing_delay = firing_delay
        self.hidden_weights = np.random.random((hidden_neurons, input_neurons))  # Generate random weights for the hidden layer
        self.output_weights = np.random.random((output_neurons, hidden_neurons))  # Generate random weights for the output layer
        self.hidden_layer = [Neuron(weights, threshold_hidden, firing_delay) for weights in self.hidden_weights]  # Create neurons for the hidden layer
        self.output_layer = [Neuron(weights, threshold_output, firing_delay) for weights in self.output_weights]  # Create neurons for the output layer

    def forward(self, inputs):
        hidden_outputs = self.activate_layer(inputs, self.hidden_layer)  # Activate the hidden layer
        output_outputs = self.activate_layer(hidden_outputs, self.output_layer)  # Activate the output layer
        return output_outputs

    def activate_layer(self, inputs, layer):
        activations = []
        for neuron in layer:
            neuron.integrate_fire(inputs)  # Integrate and fire the neuron
            if neuron.is_ready_to_fire().any():  # Check if the neuron is ready to fire
                activations.append(1)  # Add 1 to the activations if ready to fire
            else:
                activations.append(0)  # Add 0 to the activations if not ready to fire
        return np.array(activations)  # Convert the activations to a numpy array

In [5]:
#We define the number of neurons per layer.
input_neurons = 2
hidden_neurons = 4
output_neurons = 2

#We define the firing delay and threshold for the neurons.
firing_delay = 2
threshold_hidden = 0.5
threshold_output = 0.5

#We define the input data.
inputs = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 1.0], [0.0, 0.0]])

#We create the neural network with the specified architecture and parameters,
network = NeuralNetwork(input_neurons, hidden_neurons, output_neurons, threshold_hidden, threshold_output, firing_delay)

#and test the network.
outputs = network.forward(inputs)
print(outputs)

[0 0]
